# III) Création d'une ontologie

Pour ce faire, nous utiliserons le package [owlready2](https://linuxfr.org/news/owlready-un-module-python-pour-manipuler-les-ontologies-owl) qui permet de créer des ontologies en Python.
Les données seront extraite de la base de données de WikiData, nous interrogerons cette base de données par l'API mis à disposition par la librairie [wikibaseintegrator](https://github.com/LeMyst/WikibaseIntegrator#execute-sparql-queries).

Commençons par installer les packages nécessaires

In [51]:
# !pip install owlready2
# !pip install wikibaseintegrator

Récupérons les classes et sous-classes de la classe Animalia du fichier `animals_id_name_subClass_sub2Class.csv` que nous avons créé précédemment. Nous avons décité de se limiter à deux sous-classes pour des raisons de temps de récupération des informations de l'API.
De plus, nous rencontrons un problème quand les sous classes ne sont pas définies dans WikiData, nous avons donc décidé de ne pas les prendre en compte, et de créer une sous-classe d'animalia appelé sans famille.

In [1]:
import pandas as pd

df = pd.read_csv("animals_id_name_subClass_sub2Class.csv")

tableau_animaux = df['animalLabel'].values.tolist()

for i in range(len(tableau_animaux)):
    tableau_animaux[i] = tableau_animaux[i].replace(" ", "_")

# Sélectionner les colonnes spécifiques
selected_columns = ["subclass2Label", "subclassLabel"]
selected_data = df[selected_columns]

# Transformer les colonnes en un tableau de la forme [[subclass2Label], [subclassLabel]]
tableau = selected_data.values.T.tolist()

# Car une class à le nom d'un animal, on va le remplacer par le nom de l'animal + _class
for i in range(len(tableau[0])):
    if tableau[0][i] in tableau_animaux:
        tableau[0][i] = tableau[0][i].replace(" ", "_") + "_class"
    if tableau[1][i] in tableau_animaux:
        tableau[1][i] = tableau[1][i].replace(" ", "_") + "_class"

# Afficher le résultat
print(tableau)

# Afficher le résultat
print(tableau_animaux)

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'penguin', 'mammal', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', 'felidae', 'felidae', '0', 'animal', '0', '0', '0', '0', '0', 'fish', '0', '0', '0', '0', '0', '0', 'food ingredient', '0', '0', '0', '0', '0', 'elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'penguin', 'elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', 'big cat', 'big cat', '0', 'bear_class', '0', '0', '0', '0', '0', 'electric fish', '0', '0', '0', '0', '0', '0', 'vegetable', '0', '0',

Nous devons modifier la structure des données pour qu'elle soit exploitable par la suite. Par exemple, nous avons un animal qui a pour sous-classe "éléphant" et "mammifère" un autre a pour sous-classe "éléphant d'Asie" et éléphant", donc nous devons créer une classe éléphant d'Asie qui découle d'éléphant qui elle-même découle de mammifère.

In [2]:
# Nous récupérons les sous-sous-classes en supprimant les 0 et les doublons
import numpy as np

# Supprimer les zéros
tableau_sans_zeros = [x for x in tableau[0] if x != '0']
# Supprimer les doublons
tableau_sans_doublons = np.unique(tableau_sans_zeros)
print(tableau_sans_doublons)

# On parcourt les sous-sous-classes récupérées et on les ajoute un tableau de la forme [[..., sous-sous-sous-classe, sous-sous-classe, sous-classe], ...]
tableau_classes = [[x] for x in tableau_sans_doublons]

for i in range(len(tableau_classes)):
    for j in range(len(tableau[0])):
        if tableau[0][j] != '0' and tableau[1][j] != '0':
            if tableau_sans_doublons[i] == tableau[0][j] and tableau[1][j] not in tableau_classes[
                i]:  # On met le 1er élément de la sous-sous-classe dans le tableau
                # On vient de trouver une sous-sous-classe, maintenant, nous allons l'ajouter à notre tableau
                tableau_classes[i].append(tableau[1][j])
            elif tableau_sans_doublons[i] == tableau[1][j] and tableau[0][j] not in tableau_classes[
                i]:  # On met le 2ème élément de la sous-sous-classe dans le tableau
                # On vient de trouver une sous-sous-classe, maintenant, nous allons l'ajouter en tête à notre tableau
                tableau_classes[i].insert(0, tableau[0][j])
            else:
                pass
        else:
            pass


def is_subset(l1, l2):
    return set(l1).issubset(l2)


descendance_classe = []
for i in range(len(tableau_classes)):
    if not any(is_subset(tableau_classes[i], tableau_classes[j]) for j in range(len(tableau_classes)) if i != j):
        descendance_classe.append(tableau_classes[i])

# Afficher le résultat
print(tableau_classes)

# Afficher le résultat
print(descendance_classe)

['animal' 'elephant' 'felidae' 'fish' 'food ingredient' 'mammal' 'penguin']
[['animal', 'bear_class'], ['mammal', 'elephant', 'asian elephant'], ['felidae', 'big cat'], ['fish', 'electric fish'], ['food ingredient', 'vegetable'], ['mammal', 'elephant'], ['penguin']]
[['animal', 'bear_class'], ['mammal', 'elephant', 'asian elephant'], ['felidae', 'big cat'], ['fish', 'electric fish'], ['food ingredient', 'vegetable'], ['penguin']]


On récupère la liste des images de l'évaluation, si elle est reconnue ou non, si oui 1, sinon 0. Et on formate le fichier pour qu'il soit exploitable par la suite.

In [3]:
image_reconnue = pd.read_csv("./reconnues.csv")

image_reconnue.head()

# Le transformer en tableau de la forme [[image, reconnue], ...]
tableau_image_reconnue = image_reconnue.values.tolist()

# Afficher le résultat
print(tableau_image_reconnue)

[['n01443537_8662', 'goldfish', 0, 1], ['n01443537_8693', 'goldfish', 0, 1], ['n01443537_8719', 'goldfish', 0, 0], ['n01443537_875', 'goldfish', 0, 1], ['n01443537_8787', 'goldfish', 0, 1], ['n01443537_8789', 'goldfish', 0, 1], ['n01443537_8818', 'goldfish', 0, 1], ['n01443537_883', 'goldfish', 0, 1], ['n01443537_8841', 'goldfish', 0, 1], ['n01443537_8868', 'goldfish', 0, 1], ['n01443537_888', 'goldfish', 0, 1], ['n01443537_891', 'goldfish', 0, 1], ['n01443537_8938', 'goldfish', 0, 1], ['n01443537_894', 'goldfish', 0, 1], ['n01443537_896', 'goldfish', 0, 1], ['n01443537_8989', 'goldfish', 0, 1], ['n01443537_9048', 'goldfish', 0, 1], ['n01443537_9069', 'goldfish', 0, 1], ['n01443537_9106', 'goldfish', 0, 1], ['n01443537_9160', 'goldfish', 0, 1], ['n01443537_9161', 'goldfish', 0, 1], ['n01443537_9204', 'goldfish', 0, 1], ['n01443537_9265', 'goldfish', 0, 0], ['n01443537_9315', 'goldfish', 0, 1], ['n01443537_9338', 'goldfish', 0, 1], ['n01443537_9347', 'goldfish', 0, 1], ['n01443537_9364'

Retrouvons le xml de chaque image et récupérons les informations qui nous intéressent.

In [4]:
import xml.etree.ElementTree as ET
import os


def get_infos_xml(nom_image_f):
    # Récupérer le nom du sous-dossier, split avant le _
    nom_sous_dossier = nom_image_f.split("_")[0]

    url = "./ILSVRC/Annotations/CLS-LOC/train/" + nom_sous_dossier + "/" + nom_image_f + ".xml"
    # Initialiser les variables
    nom_animal_f, width, height, depth = None, None, None, None

    trouve = os.path.exists(url)
    if trouve:
        # Récupérer le xml

        tree = ET.parse(url)
        root = tree.getroot()

        # Récupérer les informations
        for child in root:
            if child.tag == "object":
                for child2 in child:
                    if child2.tag == "name":
                        nom_animal_f = child2.text
                    else:
                        pass
            elif child.tag == "size":
                for child2 in child:
                    if child2.tag == "width":
                        width = child2.text
                    elif child2.tag == "height":
                        height = child2.text
                    elif child2.tag == "depth":
                        depth = child2.text
                    else:
                        pass
            else:
                pass
    else:
        nom_animal_f, width, height, depth = "", 0, 0, 0

    return nom_animal_f, width, height, depth, url, trouve


# Afficher le résultat
print(get_infos_xml("n01440764_18"))

('n01440764', '375', '500', '3', './ILSVRC/Annotations/CLS-LOC/train/n01440764/n01440764_18.xml', True)


Concernant la contrainte de la propriété possède pour les sous-classes d'Animalia, c'est un concept avancé dans OWL appelé "Restrictions sur les propriétés". Pour répondre au mieux à cette contrainte, nous allons utiliser AllDisjoint et AllDifferent.

In [5]:
from owlready2 import Thing, AllDisjoint
import owlready2 as owl

# Création d'un URI
onto = owl.get_ontology("http://example.com/animalia.owl")

# Définir la classe racine Animalia
with onto:
    class Animalia(Thing):
        pass

# Définir les sous-classes
with onto:
    for i in range(len(descendance_classe)):
        for j in range(len(descendance_classe[i])):
            if j == 0:  # La sous-classe descent de animalia
                nom_classe = descendance_classe[i][j].replace(" ", "_")  # Remplacer les espaces par des underscores
                # Créer une classe dynamiquement
                classe = type(nom_classe, (Animalia,), {})
            else:  # c'est une sous-classe du précédent
                nom_classe = descendance_classe[i][j].replace(" ", "_")  # Remplacer les espaces par des underscores
                # Créer une classe dynamiquement
                classe = type(nom_classe, (classe,), {})

# Définir les classes orphelines
with onto:
    class SansFamille(Animalia):
        pass

# Définition de la classe image avec son nom et si elle est reconnue ou non
with onto:
    class Image(Thing):
        pass


    class Reconnue(Image >> bool):
        pass


    class URL(Image >> str):
        pass


    class Width(Image >> int):
        pass


    class Height(Image >> int):
        pass


    class Depth(Image >> int):
        pass

# Définir les exclusions, chaque sous tableau du tableau descendance_classe contient les sous-classes d'une classe, donc on va créer une exclusion pour chaque sous tableau
with onto:
    for i in range(len(descendance_classe)):
        aux = []
        for j in range(len(descendance_classe)):
            if i != j:
                for k in range(len(descendance_classe[j])):
                    aux.append(descendance_classe[j][k].replace(" ", "_"))
        print(aux)
        for n in range(len(descendance_classe[i])):
            aux.append(descendance_classe[i][n].replace(" ", "_"))
            AllDisjoint(aux)

# Définir les différentes propriétés
with onto:
    class CaracteristiqueMorphologique(Thing):
        pass


    class Patte(CaracteristiqueMorphologique):
        pass


    class Museau(CaracteristiqueMorphologique):
        pass


    class Aile(CaracteristiqueMorphologique):
        pass


    class Bec(CaracteristiqueMorphologique):
        pass


    class Queue(CaracteristiqueMorphologique):
        pass


    class Poil(CaracteristiqueMorphologique):
        pass


    class Plume(CaracteristiqueMorphologique):
        pass

# Pour définir les associations, on utilise la propriété contient, qui permet de relier une image à un animal, et on lui précisera si l'image a bien été reconnue, avec la propriété possède, qui permet de relier un animal à une caractéristique morphologique.
with onto:
    class Contient(owl.ObjectProperty):
        domain = [Image]
        range = [Animalia]
        pass


    class Possede(owl.ObjectProperty):
        domain = [Animalia]
        range = [CaracteristiqueMorphologique]
        pass

# On ajoute les animaux à leur classe
for i in range(len(tableau[-1])):
    if tableau[-1][i] == '0':
        nom_animal = tableau_animaux[i].replace(" ", "_")  # Remplacer les espaces par des underscores
        print("L'animal", nom_animal, "n'a pas été reconnu")
        classe = onto["SansFamille"]
        nouvel_animal = classe(nom_animal)
    else:
        nom_classe = tableau[-1][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        nom_animal = tableau_animaux[i].replace(" ", "_")  # Remplacer les espaces par des underscores
        # Accédez à la classe spécifique
        classe = onto[nom_classe]
        # Créez une nouvelle instance de cette classe
        nouvel_animal = classe(nom_animal)

# On ajoute les images à notre ontologie.
for i in range(len(tableau_image_reconnue)):
    nom_image = tableau_image_reconnue[i][0].replace(" ", "_")  # Remplacer les espaces par des underscores
    image_instance = Image(nom_image)
    if tableau_image_reconnue[i][3] == 1:
        image_instance.Reconnue.append(True)
    else:
        image_instance.Reconnue.append(False)

    # Ajout de l'URL, de la largeur, de la hauteur et de la profondeur de l'image
    _, width, height, depth, url, trouve = get_infos_xml(tableau_image_reconnue[i][0])

    if trouve:
        image_instance.URL.append(url)
        image_instance.Width.append(width)
        image_instance.Height.append(height)
        image_instance.Depth.append(depth)

    if tableau_image_reconnue[i][2] == 0:
        nom_sous_classe = "SansFamille"
    else:
        nom_sous_classe = tableau_image_reconnue[i][2].replace(" ", "_")

    nom_classe = tableau_image_reconnue[i][1].replace(" ", "_")

    nom_sous_classe = onto[nom_sous_classe]

    classe_instance = onto.search_one(type=nom_sous_classe, iri="*" + nom_classe)

    if classe_instance is None:
        classe_instance = nom_sous_classe(nom_classe)

    image_instance.Contient.append(classe_instance)

onto.save(file="3_animalia.owl", format="rdfxml")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




['mammal', 'elephant', 'asian_elephant', 'felidae', 'big_cat', 'fish', 'electric_fish', 'food_ingredient', 'vegetable', 'penguin']
['animal', 'bear_class', 'felidae', 'big_cat', 'fish', 'electric_fish', 'food_ingredient', 'vegetable', 'penguin']
['animal', 'bear_class', 'mammal', 'elephant', 'asian_elephant', 'fish', 'electric_fish', 'food_ingredient', 'vegetable', 'penguin']
['animal', 'bear_class', 'mammal', 'elephant', 'asian_elephant', 'felidae', 'big_cat', 'food_ingredient', 'vegetable', 'penguin']
['animal', 'bear_class', 'mammal', 'elephant', 'asian_elephant', 'felidae', 'big_cat', 'fish', 'electric_fish', 'penguin']
['animal', 'bear_class', 'mammal', 'elephant', 'asian_elephant', 'felidae', 'big_cat', 'fish', 'electric_fish', 'food_ingredient', 'vegetable']
L'animal indigo_bunting n'a pas été reconnu
L'animal tree_frog n'a pas été reconnu
L'animal garter_snake n'a pas été reconnu
L'animal agama n'a pas été reconnu
L'animal goldfish n'a pas été reconnu
L'animal bald_eagle n'a pa

La liste des caractéristiques que nous devons ajouter manuellement regroupe de manière non exhaustive :
- Patte
- Museau
- Aile
- Aile
- Bec
- Queue
- Poil
- Plume
- ...

Nous pouvons les ajouter à la main de la manière suivante :
```python
class Mammifere(Animalia):
        equivalent_to = [Animalia & possede.max(2, Aile) & possede.max(1, Bec) & possede.max(1, Queue) & possede.max(4, Patte) & possede.min(2, Patte) & possede.max(1, Museau) & possede.min(1, Poil)]
        pass
```
Nous pouvons décrire plus précisément les caractéristiques des animaux :
```python
my_oiseau.bec = ["court et pointu"]
```

On enregistre notre ontologie dans un fichier .owl

In [ ]:
# onto.save(file="3_animalia.owl", format="rdfxml")

L'ontologie est visible dans le fichier suivant : [3_animalia](./3_animalia.owl).